In [10]:
#--------------------------------------#
# Simulador de Substituição de Páginas #
# utilizando os algoritmos LRU e FIFO  #
# Autor: Leandro Santos                #
# Data: 09/06/2022                     #
#--------------------------------------# 

#Importar Bibliotecas
import pandas as pd
import numpy as np
import math
from termcolor import colored
from IPython.display import clear_output
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
#Parâmtros de Entrada
#fila = pd.read_csv('./arquivo.csv', sep=";") #Leitura de arquivo local
#Abrir arquivo pelo google Drive
from google.colab import drive 
drive.mount('/content/gdrive')
fila=pd.read_csv('gdrive/My Drive/Sistemas operacionais Avançado/input.csv', sep=";")

Mounted at /content/gdrive


In [3]:
#Funções
def verificaPageFault(pagina,listaPaginas): #Verifica se a pagina informada está alocada na MP
    return np.isnan(listaPaginas.MP[pagina]) 
def getFirstMP (memoriaPrincipal): #Retorna o Primeiro Frame Livre da MP
    first_empty_row = memoriaPrincipal.index[memoriaPrincipal.iloc[:, 0].isnull().values]
    return first_empty_row[0]
def verifMP (memoriaPrincipal): #Retorna a quantidade de espaço livre na MP
    return memoriaPrincipal.Pagina.isna().sum()
    


In [4]:
#FIFO
def fifo():
  #Listas e Variaveis
  tamanhoMP = int(input("Digite o tamanho da memoria principal: "))
  listaPaginas = pd.DataFrame([],index=pd.unique(fila.Pagina), columns=['MP'])
  listaPaginas = listaPaginas.sort_index()
  memoriaPrincipal = pd.DataFrame([], index=range(0,tamanhoMP), columns=['Pagina'])
  lista_substituicao = []
  logHeader = memoriaPrincipal.index.values.tolist()
  logHeader.extend(['Pagina Sub','Page Fault','Tempo'])
  log = pd.DataFrame([], index=logHeader)
  tempo = 0
  countPageFault = 0
  for pagina_atual,operacao in zip(fila.Pagina, fila.Operacao):
      pageFault = verificaPageFault(pagina_atual,listaPaginas) #Verifica se está alocada na MP
      if pageFault:        
          if verifMP(memoriaPrincipal) != 0 : #Verifica se a memoria principal não está lotada
              memoriaAlocar = getFirstMP(memoriaPrincipal) #Pega a primeira memoria disponivel
              memoriaPrincipal.Pagina[memoriaAlocar] = pagina_atual #Aloca a pagina na MP
              listaPaginas.MP[pagina_atual] = memoriaAlocar #Referencia a MP na tabela de paginas
              lista_substituicao.append(pagina_atual)            
          else:
              #Processo de substituição de Paginas na Memória
              pagina_substituir = lista_substituicao.pop(0)                      
              listaPaginas.MP[pagina_substituir] = np.nan #Remove a pagina a ser substituida da tabela de paginas
              mp_subistituida = memoriaPrincipal[memoriaPrincipal['Pagina']==pagina_substituir].index.values #Pega a posicao da memoria a ser substituida
              mp_subistituida = mp_subistituida[0]
              memoriaPrincipal.Pagina[mp_subistituida] = pagina_atual #Aloca a pagina na MP
              listaPaginas.MP[pagina_atual] = mp_subistituida #Referencia a MP na tabela de paginas
              lista_substituicao.append(pagina_atual)     
          countPageFault += 1

      logTime = []
      logTime.extend(memoriaPrincipal.Pagina)
      logTime.append(lista_substituicao[0])
      logTime.append(pageFault)
      logTime.append(tempo)
      log.insert((log.shape[1]),pagina_atual,logTime,True)
      tempo += 1

  print("")
  print("############# FILA #############",sep='\n')
  print(fila.Pagina) 
  print("")   
  print("############# LOG #############",sep='\n')
  print(log)
  print("")
  print(colored('Quantidade total de Page Fault: '+str(countPageFault),'red'))
  print("")
  print("############# M.P #############",sep='-')
  print(memoriaPrincipal)
  print("")
  print("##### Tabela de Paginas #####",sep='\n')
  print(listaPaginas)
  input("Pressione Enter para voltar ao Menu Principal"),

In [5]:
#LRU
def lru():
  #Listas e Variaveis
  tamanhoMP = int(input("Digite o tamanho da memoria principal: "))
  listaPaginas = pd.DataFrame([],index=pd.unique(fila.Pagina), columns=['MP'])
  listaPaginas = listaPaginas.sort_index()
  memoriaPrincipal = pd.DataFrame([], index=range(0,tamanhoMP), columns=['Pagina'])
  lista_substituicao = []
  logHeader = memoriaPrincipal.index.values.tolist()
  logHeader.extend(['Pagina Sub','Page Fault','Tempo'])
  log = pd.DataFrame([], index=logHeader)
  tempo = 0
  countPageFault = 0
  for pagina_atual,operacao in zip(fila.Pagina, fila.Operacao):
      pageFault = verificaPageFault(pagina_atual,listaPaginas) #Verifica se está alocada na MP
      if pageFault:        
          if verifMP(memoriaPrincipal) != 0 : #Verifica se a memoria principal não está lotada
              memoriaAlocar = getFirstMP(memoriaPrincipal) #Pega a primeira memoria disponivel
              memoriaPrincipal.Pagina[memoriaAlocar] = pagina_atual #Aloca a pagina na MP
              listaPaginas.MP[pagina_atual] = memoriaAlocar #Referencia a MP na tabela de paginas
              lista_substituicao.append(pagina_atual)            
          else:
              #Processo de substituição de Paginas na Memória
              pagina_substituir = lista_substituicao.pop(0)                      
              listaPaginas.MP[pagina_substituir] = np.nan #Remove a pagina a ser substituida da tabela de paginas
              mp_subistituida = memoriaPrincipal[memoriaPrincipal['Pagina']==pagina_substituir].index.values #Pega a posicao da memoria a ser substituida
              mp_subistituida = mp_subistituida[0]
              memoriaPrincipal.Pagina[mp_subistituida] = pagina_atual #Aloca a pagina na MP
              listaPaginas.MP[pagina_atual] = mp_subistituida #Referencia a MP na tabela de paginas
              lista_substituicao.append(pagina_atual)     
          countPageFault += 1
      else:      
        lista_substituicao.remove(pagina_atual)
        lista_substituicao.append(pagina_atual)
      logTime = []
      logTime.extend(memoriaPrincipal.Pagina)
      logTime.append(lista_substituicao[0])
      logTime.append(pageFault)
      logTime.append(tempo)
      log.insert((log.shape[1]),pagina_atual,logTime,True)
      tempo += 1

  print("")
  print("############# FILA #############",sep='\n')
  print(fila.Pagina) 
  print("")   
  print("############# LOG #############",sep='\n')
  print(log)
  print("")
  print(colored('Quantidade total de Page Fault: '+str(countPageFault),'red'))
  print("")
  print("############# M.P #############",sep='-')
  print(memoriaPrincipal)
  print("")
  print("##### Tabela de Paginas #####",sep='\n')
  print(listaPaginas)
  input("Pressione Enter para voltar ao Menu Principal")

In [9]:
def main():
    app = True
    while app:
      clear_output(wait=True)
      #Tela Inicial
      print('SIMULADOR DE SUBSTITUIÇÃO DE PÁGINAS')
      print ("""
          1.FIFO
          2.LRU
          3.Exit/Quit
          """)
      opcao = int(input('Digite o numero da opção desejada: '))
      if opcao == 1 :
        fifo()
        clear_output(wait=True)
      else:
        if opcao == 2 :
          lru()
          clear_output(wait=True)
        else:
          if opcao == 3 :
            app = False
          else:
            print('Opção Invalida, digite a opção de acordo com o menu!')       

if __name__== "__main__" :
  main()    

SIMULADOR DE SUBSTITUIÇÃO DE PÁGINAS

          1.FIFO
          2.LRU
          3.Exit/Quit
          
Digite o numero da opção desejada: 3
